In [ ]:
import os, shutil
from keras import layers, models, optimizers
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, smart_resize
from keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint
from keras.applications import VGG16
import matplotlib.pyplot as plt

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

In [ ]:
labels = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
labels.head()

In [ ]:
labels['label']=labels['label'].astype(str)

In [ ]:
train_dir = '../input/cassava-leaf-disease-classification/train_images'
test_dir = '../input/cassava-leaf-disease-classification/test_images'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./225,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   validation_split=0.3,
                                   horizontal_flip=True,)


test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe=labels,
                                                    directory=train_dir,
                                                    subset='training',
                                                    x_col="image_id",
                                                    y_col="label",
                                                    shuffle=True,
                                                    target_size=(150,150),
                                                    batch_size=32,
                                                    class_mode='categorical')

valid_generator = train_datagen.flow_from_dataframe(dataframe=labels,
                                                    directory=train_dir,
                                                    subset='validation',
                                                    x_col="image_id",
                                                    y_col="label",
                                                    shuffle=True,
                                                    target_size=(150,150),
                                                    batch_size=32,
                                                    class_mode='categorical')

In [ ]:
# conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

In [ ]:
model = models.Sequential()
# model.add(conv_base)
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(),
             metrics=['acc'])

In [ ]:
callback_list = [
    EarlyStopping(monitor='val_acc',patience=2),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1),
    ModelCheckpoint(filepath='{epoch:02d}-{val_acc:.4f}.hdf5', monitor='val_loss',save_best_only=True),
]

In [ ]:
history = model.fit(train_generator,steps_per_epoch=30,
                    validation_data=valid_generator,validation_steps=30,
                    callbacks=callback_list, epochs=100)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc)+1)

In [ ]:
plt.figure()

plt.plot(epochs, acc, 'bo', label='Train Acc')
plt.plot(epochs, val_acc, 'b', label='Validation Acc')
plt.title('Training and validation Acc')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Train Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation Loss')
plt.legend()

plt.show()

In [ ]:
submission = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")

In [ ]:
submission.head()

In [ ]:
img = load_img("../input/cassava-leaf-disease-classification/test_images/2216849948.jpg")
img = img_to_array(img)
img = smart_resize(img, (150,150))
img = tf.reshape(img, (-1, 150, 150, 3))

In [ ]:
pred = model.predict(img/255.)
pred = np.argmax(pred)
submission_result = pd.DataFrame({'image_id' : submission.image_id, 'label' : pred})

In [ ]:
submission_result.to_csv('submission.csv', index=False)

In [ ]:
print("Competetion Complete!!")